In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Package

In [ ]:
!pip install q keras==2.3.1
!pip install q tensorflow-gpu==2.1.0
!pip install q tensorflow==2.1.0

In [ ]:
import matplotlib as mpl
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
%matplotlib inline
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
print(tf.__version__)
print(keras.__version__)

# Set Variables

In [ ]:
height = 32
width = 32
channels = 3
batch_size = 32
epochs = 50
num_classes = 10

# Function

In [ ]:
def PreprocessData(X_Train, Y_Train, X_Test, Y_Test, nums_class):
    #Normalize picture
    X_Train = X_Train.astype('float32')/255.
    X_Test = X_Test.astype('float32')/255.
    #split training data into training and validation set
    X_train, X_valid, Y_train, Y_valid = train_test_split(X_Train, Y_Train, test_size=0.2, random_state=7, shuffle= True)
    Y_train = keras.utils.to_categorical(Y_train)
    Y_valid = keras.utils.to_categorical(Y_valid)
    Y_Test = keras.utils.to_categorical(Y_Test)
    return ((X_train, Y_train), (X_valid, Y_valid), (X_Test, Y_Test))

In [ ]:
def Aug_Data(X_train, Y_train, X_valid, Y_valid, batch_size):
    train_datagen = keras.preprocessing.image.ImageDataGenerator(
    shear_range = 0.2,
    zoom_range = 0.2,
    horizontal_flip = True,
    )
    validation_datagen = keras.preprocessing.image.ImageDataGenerator()
    train_generator = train_datagen.flow(X_train, Y_train, batch_size=batch_size)
    valid_generator = validation_datagen.flow(X_valid, Y_valid, batch_size=batch_size)
    return train_generator, valid_generator

In [ ]:
def BuildModel(filter_number, layer_number, dropout, optimization):
    model = keras.models.Sequential()
    model.add(keras.layers.Conv2D(filters=filter_number, 
                                  kernel_size=(3,3), 
                                  padding='same', 
                                  activation='relu',
                                  input_shape=[width, height, channels]))
    model.add(keras.layers.BatchNormalization())
    model.add(keras.layers.Dropout(dropout)),
    model.add(keras.layers.MaxPool2D(pool_size=2))
    for i in range(layer_number):
        model.add(keras.layers.Conv2D(filters=filter_number*(2**(i+1)), 
                                  kernel_size=(3,3), 
                                  padding='same', 
                                  activation='relu',
                                  input_shape=[width, height, channels]))
        model.add(keras.layers.BatchNormalization())
        keras.layers.Dropout(dropout)
        model.add(keras.layers.MaxPool2D(pool_size=2))
    model.add(keras.layers.Flatten())
    model.add(keras.layers.Dropout(dropout))
    model.add(keras.layers.Dense(256))
    model.add(keras.layers.ReLU())
    model.add(keras.layers.Dense(num_classes, activation='softmax'))
    model.compile(loss='categorical_crossentropy', optimizer=optimization, metrics=['acc'])
    return model

In [ ]:
def training_Process(train_generator, valid_generator, MLName):
    earlystop = keras.callbacks.EarlyStopping(monitor='loss', patience=10, restore_best_weights=True, verbose=1)
    cp = keras.callbacks.ModelCheckpoint(filepath = MLName,save_best_only = True,verbose=1)
    lr = keras.callbacks.ReduceLROnPlateau(monitor='loss', factor=0.2, patience=5, min_delta=0.0001)
    callbacks = [earlystop, cp, lr]
    history = model.fit(train_generator,
                        steps_per_epoch = len(train_generator) // batch_size,
                        epochs = epochs,
                        callbacks = callbacks,
                        validation_data = valid_generator,
                        validation_steps = len(valid_generator) // batch_size)
    return history

In [ ]:
def visulize_map(model, img):
    layer_names = [layer.name for layer in model.layers]
    layer_outputs = [layer.output for layer in model.layers]
    new_model=keras.models.Model(inputs=model.inputs,outputs=layer_outputs)
    img = np.expand_dims(img, axis=0)
    feature_maps=new_model.predict(img)
    
    for layer_name, feature_map in zip(layer_names, feature_maps):
        if len(feature_map.shape) == 4:
            n_features = feature_map.shape[-1]
            size       = feature_map.shape[ 1]
            display_grid = np.zeros((size, size * n_features))

            for i in range(n_features):
                x  = feature_map[0, :, :, i]
                x -= x.mean()
                x /= x.std ()
                x *=  64
                x += 128
                x  = np.clip(x, 0, 255).astype('uint8')
                display_grid[:, i * size : (i + 1) * size] = x
            scale = 20. / n_features
            plt.figure( figsize=(scale * n_features, scale) )
            plt.title ( layer_name )
            plt.grid  ( False )
            plt.imshow( display_grid, aspect='auto', cmap='viridis' )

# Preprocess Data

In [ ]:
cifar100 = keras.datasets.cifar100.load_data()
(x_train_all, y_train_all), (x_test, y_test) = cifar100
(X_train, Y_train), (X_valid, Y_valid), (X_test, Y_test) = PreprocessData(x_train_all, y_train_all, x_test, y_test, num_classes)
train_generator, valid_generator = Aug_Data(X_train, Y_train, X_valid, Y_valid, batch_size)
print(X_train.shape)

# Build Model

In [ ]:
model = BuildModel(32, 2, 0.25, 'adam')
model.summary()

# Training Process

In [ ]:
history = training_Process(train_generator, valid_generator, 'cifar100.h5')

In [ ]:
model = keras.models.load_model('./cifar100.h5')

In [ ]:
model.evaluate(X_test, Y_test)

In [ ]:
visulize_map(model, X_test[0])

# Drawing Picture

In [ ]:
def plots_learning_curves(history, label, epochs, min_value, max_value, title):
    data = {}
    data[label] = history.history[label]
    data['val_'+label] = history.history['val_'+label]
    pd.DataFrame(data).plot(figsize=(8, 5))
    plt.grid(True)
    plt.title (title)
    plt.axis([0, epochs, min_value, max_value])
    plt.show()
plots_learning_curves(history, 'acc', epochs, 0, 1, 'cifar100')
plots_learning_curves(history, 'loss', epochs, 0, 10, 'cifar100')

# Optimize the network

# Cifar10 and Cifar100 Optimization

In [ ]:
cifar10 = keras.datasets.cifar10.load_data()
(x_train_all, y_train_all), (x_test, y_test) = cifar10
(X_train, Y_train), (X_valid, Y_valid), (X_test, Y_test) = PreprocessData(x_train_all, y_train_all, x_test, y_test, num_classes)
train_generator, valid_generator = Aug_Data(X_train, Y_train, X_valid, Y_valid, batch_size)
print(X_train.shape)

In [ ]:
model = BuildModel(64, 3, 0.25, 'adam')
model.summary()

In [ ]:
history = training_Process(train_generator, valid_generator, 'best-cifar10.h5')

In [ ]:
model = keras.models.load_model('./cifar10-1.h5')
model.evaluate(X_test, Y_test)

In [ ]:
parameter_labels = ['dataset', 'layers', 'filter_number', 'dropout', 'optimizer', 'test_accuracy']
value = [['cifar10', 2, 32, 0.25, 'adam', 0.6465], 
         ['cifar10', 2, 64, 0.25, 'adam', 0.5972], 
         ['cifar10', 3, 64, 0.25, 'adam', 0.6522],
         ['cifar10', 3, 32, 0.35, 'adam', 0.6211],
         ['cifar10', 2, 32, 0.25, 'SGD', 0.5799],
         ['cifar100', 2, 32, 0.25, 'adam', 0.2279], 
         ['cifar100', 3, 32, 0.25, 'adam', 0.2431], 
         ['cifar100', 3, 64, 0.25, 'adam', 0.2128],
         ['cifar100', 3, 32, 0.35, 'adam', 0.2272],
         ['cifar100', 3, 32, 0.25, 'SGD', 0.1880],]
pd.DataFrame(data=value, columns=parameter_labels)